In [1]:
%pwd

'c:\\Users\\ritik\\data science\\data analyat\\cerina_task\\demo\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [7]:
from dataclasses  import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from Homestayes_data.constants import *
from Homestayes_data.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from Homestayes_data import logger
from Homestayes_data.utils.common import get_size

class DataIngestion:
    def __init__(self,config:DataIngestionConfig): 
        self.config=config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following data: \n{headers} ")
        else:
            logger.info(f"file already exists of size :{get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try :
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-05-06 18:14:21,745: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-06 18:14:21,747: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-06 18:14:21,751: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-06 18:14:21,752: INFO: common: created directory at: artifacts]
[2024-05-06 18:14:21,753: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-06 18:14:32,938: INFO: 308877453: artifacts/data_ingestion/data.zip download! with following data: 
Connection: close
Content-Length: 31030310
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5a3239d4f2155bf1c19c09107a159f020c35456e64e2f3db20603dc6dec6cbc7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: ED9E:210E34:13B142:17A1C1:6638D0A7
Accept-Ranges: bytes
Date: Mon, 